In [ ]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

In [ ]:
summarizer = pipeline('summarization')

In [8]:
URL = "https://towardsdatascience.com/a-bayesian-take-on-model-regularization-9356116b6457"
# URL = "https://hackernoon.com/will-the-game-stop-with-gamestop-or-is-this-just-the-beginning-2j1x32aa"

In [9]:
r = requests.get(URL)

In [10]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [11]:
ARTICLE

'Sign up Sign In Sign up Sign In Member-only story A Bayesian Take On Model Regularization Ryan Sander Follow Towards Data Science -- 1 Share I’m currently reading “How We Learn” by Stanislas Dehaene. First off, I cannot recommend this book enough to anyone interested in learning, teaching, or AI. One of the main themes of this book is explaining the neurological and psychological bases of why humans are so good at learning things quickly and with great sample-efficiency, i.e. given only a limited amount of experience¹. One of Dehaene’s main arguments of why humans can learn so effectively is because we are able to reduce the complexity of models we formulate of the world. In accordance with the principle of Occam’s Razor², we find the simplest model possible that explains the data we experience, rather than opting for more complicated models. But why do we do this, even from birth¹? One argument is that, contrary to the frequentist view in child psychology (the belief that babies lear

In [12]:
max_chunk = 500

In [13]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')
sentences = ARTICLE.split('<eos>')

In [14]:
sentences[0]

'Sign up Sign In Sign up Sign In Member-only story A Bayesian Take On Model Regularization Ryan Sander Follow Towards Data Science -- 1 Share I’m currently reading “How We Learn” by Stanislas Dehaene.'

In [15]:
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [16]:
chunks

['Sign up Sign In Sign up Sign In Member-only story A Bayesian Take On Model Regularization Ryan Sander Follow Towards Data Science -- 1 Share I’m currently reading “How We Learn” by Stanislas Dehaene.  First off, I cannot recommend this book enough to anyone interested in learning, teaching, or AI.  One of the main themes of this book is explaining the neurological and psychological bases of why humans are so good at learning things quickly and with great sample-efficiency, i. e.  given only a limited amount of experience¹.  One of Dehaene’s main arguments of why humans can learn so effectively is because we are able to reduce the complexity of models we formulate of the world.  In accordance with the principle of Occam’s Razor², we find the simplest model possible that explains the data we experience, rather than opting for more complicated models.  But why do we do this, even from birth¹?  One argument is that, contrary to the frequentist view in child psychology (the belief that ba

In [17]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

Token indices sequence length is longer than the specified maximum sequence length for this model (638 > 512). Running this sequence through the model will result in indexing errors


In [18]:
res[0]

{'summary_text': 'a Bayesian take on model regularization Ryan Sander Follows Towards Data Science . the book is based on the principle of Occam’s Razor2 . one argument is that we are already imparted with prior beliefs about the world when we are born .'}

In [19]:
text = ' '.join([summ['summary_text'] for summ in res])

In [20]:
with open('blogsummary.txt', 'w') as f:
    f.write(text)